In [86]:
# set-up
import pandas as pd
import urllib
import requests
import re
from bs4 import BeautifulSoup
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVDpp, accuracy
from sklearn import preprocessing

import seaborn as sns

# Import Dataset

In [87]:
user_url = "https://raw.githubusercontent.com/mkivenson/" + \
"Recommender-Systems/master/Video%20Game%20Recommendations/steam-200k.csv"
user_df = pd.read_csv(user_url, 
                      header = None,
                      usecols = list(range(4)),
                      names = ['user_id','game_title','action','hours_played'])
user_df.head(n=10)

,user_id,game_title,action,hours_played
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0
5,151603712,Spore,play,14.9
6,151603712,Fallout New Vegas,purchase,1.0
7,151603712,Fallout New Vegas,play,12.1
8,151603712,Left 4 Dead 2,purchase,1.0
9,151603712,Left 4 Dead 2,play,8.9


In [88]:
user_df = (pd.pivot_table(user_df, 
                      index = ['user_id','game_title'], 
                      columns='action', 
                      values='hours_played')
        .reset_index()
        .fillna(0)
        .rename(index = str, columns = {'play': 'hours_played', 'purchase':'purchased'}))
print(len(user_df))
matrix = (user_df.pivot(index = 'user_id', columns = 'game_title', values = 'purchased')
                    .fillna(int(0))
                    .reset_index())
matrix

128804


game_title,user_id,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,...,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,samurai_jazz,the static speaks my name,theHunter,theHunter Primal
0,5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,103360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,144736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,181212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,298950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,299153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,381543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_df_full = pd.melt(user_df_full, id_vars="user_id")
user_df_full

In [54]:
user_df.head()

action,user_id,game_title,hours_played,purchased
0,5250,Alien Swarm,4.9,1.0
1,5250,Cities Skylines,144.0,1.0
2,5250,Counter-Strike,0.0,1.0
3,5250,Counter-Strike Source,0.0,1.0
4,5250,Day of Defeat,0.0,1.0


In [82]:
reader = Reader()
user_result = Dataset.load_from_df(user_df_full[['user_id', 'game_title', 'value']], reader)
train, test = train_test_split(user_result, test_size=0.25)
algo_SVDpp = SVDpp()
algo_SVDpp.fit(train)
predictions = algo_SVDpp.test(test)
from surprise import accuracy
accuracy.rmse(predictions)

KeyboardInterrupt: 

In [58]:
pd.DataFrame(predictions)

,uid,iid,r_ui,est,details
0,112865325,Dethroned!,1.0,1.053349,{'was_impossible': False}
1,170172944,Elsword,1.0,1.000000,{'was_impossible': False}
2,80239919,Grand Theft Auto IV,1.0,1.000000,{'was_impossible': False}
3,59825286,Game Dev Tycoon,1.0,1.010708,{'was_impossible': False}
4,12130111,The Escapists The Walking Dead,1.0,1.000000,{'was_impossible': False}
5,55367062,The Elder Scrolls V Skyrim - Hearthfire,1.0,1.000000,{'was_impossible': False}
6,116564064,DARK SOULS II,1.0,1.010525,{'was_impossible': False}
7,169902951,The Gate,1.0,1.024205,{'was_impossible': False}
8,150789931,The Guild Gold Edition,1.0,1.038335,{'was_impossible': False}
9,49462664,Half-Life 2 Episode One,1.0,1.001632,{'was_impossible': False}


In [ ]:
def get_game_details(game):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    #make game title into url-friendly format
    game_url = re.sub('\s+','-',game).lower()
    
    #web-scraping request
    URL = "https://www.metacritic.com/game/pc/" + game_url + "/details"
    request = urllib.request.Request(URL, None, headers)
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page)
    summary = soup.select("div.summary_detail.product_summary .data")[0].text

    #get publisher
    publisher = re.sub('\n| ','',soup.select("li.summary_detail.publisher .data a")[0].text).lower()

    #get index of details
    details = [x.text for x in soup.select("#main > div.product_details th")]
    for idx, x in enumerate(details):
        if x  == 'Genre(s):':
            genre_position = idx
        if x  == 'Developer:':
            dev_position = idx

    #get genre
    genre = " ".join(re.findall('\w+-?\w+', 
                                soup.select("#main > div.product_details td")[genre_position].text)).lower()
    #get developer
    developer = " ".join(re.findall('\w+-?\w+', 
                                    soup.select("#main > div.product_details td")[dev_position].text)).lower()

    #get user score
    user_score = float(soup.select('.userscore_wrap.feature_userscore .metascore_w')[0].text)

    #get amount of ratings
    no_ratings = int(re.findall('\d+', soup.select('.userscore_wrap.feature_userscore .count a')[0].text)[0])
    return (game, summary, publisher, developer, genre, user_score, no_ratings)

In [ ]:
games_df = pd.DataFrame(columns = ['game_title','summary','publisher','developer','genre', 'user_score','no_ratings'])
failures = []

for game in user_df['game_title']:
    try:
        game_info = get_game_details(game)
        games_df = games_df.append(pd.DataFrame([game_info],
                                                columns = games_df.columns,
                                                ignore_index = True))
    except Exception as error:
        failures.append((game, error))
games_df.head()

game failed: arma-2
game failed: arma-2-operation-arrowhead
game failed: arma-2-operation-arrowhead-beta-(obsolete)
game failed: call-of-duty-black-ops---multiplayer
game failed: call-of-duty-modern-warfare-2---multiplayer
game failed: call-of-duty-modern-warfare-3---multiplayer
game failed: thief
game failed: thief---ghost
game failed: thief---opportunist
game failed: thief---predator
game failed: thief---the-bank-heist
game failed: thief-2
game failed: thief-deadly-shadows
game failed: thief-gold
game failed: age-of-empires-ii-hd-the-forgotten
game failed: alan-wake
game failed: alan-wake's-american-nightmare
game failed: arma-2
game failed: arma-2-operation-arrowhead
game failed: arma-2-operation-arrowhead-beta-(obsolete)
game failed: back-to-the-future-ep-1---it's-about-time
game failed: back-to-the-future-ep-2---get-tannen!
game failed: back-to-the-future-ep-3---citizen-brown
game failed: back-to-the-future-ep-4---double-visions
game failed: back-to-the-future-ep-5---outatime
game failed: deus-ex-game-of-the-year-edition
game failed: dirt-3-complete-edition
game failed: hector-ep-1
game failed: hector-ep-2
game failed: hector-ep-3
game failed: killer-is-dead
game failed: l.a.-noire
game failed: sam-&-max-301-the-penal-zone
game failed: sam-&-max-302-the-tomb-of-sammun-mak
game failed: sam-&-max-303-they-stole-max's-brain!
game failed: sam-&-max-304-beyond-the-alley-of-the-dolls
game failed: sam-&-max-305-the-city-that-dares-not-sleep
game failed: serious-sam-2
game failed: serious-sam-classic-the-first-encounter
game failed: serious-sam-classic-the-second-encounter
game failed: serious-sam-classics-revolution
game failed: serious-sam-double-d-xxl
game failed: skyrim-high-resolution-texture-pack

In [13]:
try:
    res = get_game_details('thief')
except Exception as error:
    print(error)

In [14]:
res

('thief',
 'https://www.metacritic.com/game/pc/thief/details',
 'Garrett, the Master Thief, steps out of the shadows into the City. In this treacherous place, where the Baron’s Watch spreads a rising tide of fear and oppression, his skills are the only things he can trust. Even the most cautious citizens and their best- guarded possessions are not safe from his reach. As an uprising emerges, Garrett finds himself entangled in growing layers of conflict. Lead by Orion, the voice of the people, the tyrannized citizens will do everything they can to claim back the City from the Baron’s grasp. The revolution is inevitable. If Garrett doesn’t get involved, the streets will run red with blood and the City will tear itself apart.',
 'squareenix',
 'nixxes software',
 'fantasy action adventure general fantasy',
 5.9,
 1539)